In [1]:
import pandas as pd
df = pd.read_parquet("GUIDE_train_Feature_engineered_2.parquet")

In [2]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['IncidentGrade'])
y = df['IncidentGrade']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, stratify=y, random_state=42)

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, roc_auc_score, roc_curve

# Initialize the Random Forest model
rf = RandomForestClassifier(random_state=42)

# Define a grid of hyperparameters
param_grid = {
    'n_estimators': [75, 125],
    'max_depth': [None],
    'min_samples_split': [2, 3],
    'min_samples_leaf': [1, 2]
}

# Perform grid search
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Best parameters
print(f"Best Parameters: {grid_search.best_params_}")

# Evaluate the model
y_pred = grid_search.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))

# Calculate ROC-AUC for each class
y_prob = grid_search.predict_proba(X_test)
roc_aucs = {}
for i in range(y_prob.shape[1]):
    roc_aucs[i] = roc_auc_score(y_test == i, y_prob[:, i])
    fpr, tpr, _ = roc_curve(y_test == i, y_prob[:, i])
    plt.plot(fpr, tpr, label=f'Class {i} (area = {roc_aucs[i]:.2f})')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='best')
plt.title('ROC Curves for Random Forest')
plt.show()

# Cross-validation scores
cv_scores = grid_search.cv_results_['mean_test_score']
print(f'Cross-validation scores: {cv_scores}')


Fitting 5 folds for each of 8 candidates, totalling 40 fits


C:\Users\naray\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
36 fits failed out of a total of 40.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\naray\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\naray\anaconda3\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\naray\anaconda3\Lib\site-packages\sklearn\ensemble\_forest.py", line 489, in fit
    trees = Parallel(


Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 75}


NameError: name 'accuracy_score' is not defined

In [13]:
import pickle

# Specify the filename for the model
filename = 'grid_search_parameters.pkl'

# Save the model to disk
with open(filename, 'wb') as file:
    pickle.dump(grid_search.best_params_, file)

print(f"Model saved as {filename}")

Model saved as grid_search_parameters.pkl
